In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
from numpy import exp, log
from scipy.stats import linregress
import statsmodels.api as sm

In [2]:
Bs = np.array([2/3,5/6,7/6,4/3])
topks = np.array([0.25,0.5,0.75])
sigmas = np.array([0.01,0.05,0.1])
Ns = np.array([100,300,500,800,3000])
Ntrials = 500

In [3]:
def regress(x,y):
    xdf  = pd.DataFrame({"X":x})
    X_const = sm.add_constant(xdf)
    res = sm.OLS(y, X_const).fit()
    slope = res.params["X"]
    slope_low = res.conf_int(alpha=0.05).T["X"][0]
    slope_hight = res.conf_int(alpha=0.05).T["X"][1]
    return slope, slope_low, slope_hight, res.rsquared

In [5]:
def Simulate_Normal(N, beta, sigma, topk):
    X = np.random.normal(100000, 10000, N)  # Generate X values from a normal distribution
    eps = np.random.normal(0, sigma, N)     # Generate noise for Y
    Y = X**beta * exp(eps)                  # Generate Y based on X and noise
    
    # Sort X and Y by Y values in descending order
    sorted_indices = np.argsort(-Y)  
    X_sorted = X[sorted_indices]
    Y_sorted = Y[sorted_indices]
    
    cumulative_sum_Y = np.cumsum(Y_sorted)
    total_sum_Y = np.sum(Y_sorted)
    
    topk_threshold = total_sum_Y * topk  
    selected_idx = np.argmax(cumulative_sum_Y >= topk_threshold) + 1 
    
    X_hat = X_sorted[:selected_idx]
    Y_hat = Y_sorted[:selected_idx]
    
    return regress(log(X_hat),log(Y_hat))


def Simulate_lognormal(N, beta, sigma, topk):
    X = np.random.lognormal(10,0.1,N)
    eps = np.random.normal(0,sigma,N)
    Y = X**(beta)*exp(eps)
    sorted_indices = np.argsort(-Y)  # Negative for descending order
    X_sorted = X[sorted_indices]
    Y_sorted = Y[sorted_indices]
    cumulative_sum_Y = np.cumsum(Y_sorted)
    total_sum_Y = np.sum(Y_sorted)
    topk_threshold = total_sum_Y * topk  # This is the sum threshold for the top k%
    selected_idx = np.argmax(cumulative_sum_Y >= topk_threshold) + 1  # Find the cutoff index
    X_hat = X_sorted[:selected_idx]
    Y_hat = Y_sorted[:selected_idx]
    return regress(log(X_hat),log(Y_hat))

In [6]:
df = pd.DataFrame(columns = ["Distribution","Beta","N","topk","sigma","B_hat","Bh_L","Bh_H","R2"])
for beta in Bs:
    for topk in topks:
        for sigma in sigmas:
            for N in Ns:
                for ntraisl in range(Ntrials):
                    B_hat, B_hat_low, B_hat_hight, R2 = Simulate_Normal(N, beta, sigma, topk)
                    df.loc[len(df.index)] = ['Normal', beta, N, topk, sigma, B_hat, B_hat_low, B_hat_hight, R2] 
                    B_hat, B_hat_low, B_hat_hight, R2 = Simulate_lognormal(N, beta, sigma, topk)
                    df.loc[len(df.index)] = ['Lognormal', beta, N, topk, sigma, B_hat, B_hat_low, B_hat_hight, R2]  

In [7]:
df.to_csv("results_4Ns_topk.csv")